## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-20-52-00 +0000,wsj,Opinion | This Is the Pacific Northwest on Drugs,https://www.wsj.com/opinion/this-is-the-pacifi...
1,2025-08-06-20-50-00 +0000,wsj,Stock Market Today: Trump Hikes India Tariffs;...,https://www.wsj.com/livecoverage/stock-market-...
2,2025-08-06-20-48-08 +0000,nypost,Ex-top doc at SUNY hospital sentenced for stea...,https://nypost.com/2025/08/06/us-news/ex-top-d...
3,2025-08-06-20-45-50 +0000,nyt,Our Reporter Speaks to a Ukrainian Sniper Abou...,https://www.nytimes.com/video/world/europe/100...
4,2025-08-06-20-44-43 +0000,nyt,New Zealand Air Force Mounts Dangerous Rescue ...,https://www.nytimes.com/2025/08/06/world/new-z...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2378/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,55
31,new,16
8,india,13
92,ukraine,11
97,will,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
129,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...,115
40,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...,110
310,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...,107
263,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,106
1,2025-08-06-20-50-00 +0000,wsj,Stock Market Today: Trump Hikes India Tariffs;...,https://www.wsj.com/livecoverage/stock-market-...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
129,115,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...
40,55,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...
207,50,2025-08-06-10-00-00 +0000,latimes,Chabria: Newsom vows Texas will be 'neutered' ...,https://www.latimes.com/california/story/2025-...
41,49,2025-08-06-19-43-33 +0000,nypost,NYC designer Martha Nolan’s body had no ‘evide...,https://nypost.com/2025/08/06/us-news/nyc-desi...
88,44,2025-08-06-17-46-00 +0000,wsj,Democrats are adding potentially risky Epstein...,https://www.wsj.com/politics/policy/epstein-fi...
100,40,2025-08-06-16-56-15 +0000,nypost,Italy approves $15.5 billion project to build ...,https://nypost.com/2025/08/06/world-news/italy...
48,39,2025-08-06-19-27-00 +0000,wsj,New York Times Digital Revenue Gains Help Push...,https://www.wsj.com/business/media/new-york-ti...
6,33,2025-08-06-20-39-20 +0000,nypost,Neighbors of veterinarian killed by NJ trooper...,https://nypost.com/2025/08/06/us-news/screamin...
324,32,2025-08-05-21-08-00 +0000,wsj,Opinion | China’s Z.ai and America’s Self-Defe...,https://www.wsj.com/opinion/chinas-z-ai-and-am...
181,30,2025-08-06-11-49-45 +0000,nypost,Virgin Atlantic passenger threatens to rape fl...,https://nypost.com/2025/08/06/world-news/staff...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
